In [ ]:
import numpy as np
from scipy import optimize
from scipy.stats import lognorm
from scipy.integrate import quad
from numpy import sqrt


# set economy-wide parameters

T = 20 # patent length
l0 = 590000 # minimum litigation cost
l1 = 0.15 # rate at which litigation cost increases with value at stake
qH = 1 # high type's win probability in court
qL = 0 # low type's win probability in court
phiA = 16318 # pre-grant fee + attorney costs
phiP = 5985 # post-grant fee
gr0 = 0.675
gr1 = 0.778 # grant rate before and after examiner time is doubled (from Frakes & Wasserman)
bdgt = 4130 # patent office examination budget per application
neg_fee = 1 # equals 1 if negative fixed fees are allowed
dist = 1 # distribution of cost reduction s. possible values: 0 = frechet or 1 = loglogistic
negk0 = 0 # if equal to 1, mean of R&D cost is adjusted such that it converges to k0 + k1*s as s grows, where k0 can be either positive or negative
num = 0 # 0 = count only inventions on which patents are applied for in TFP and R&D, 1 = count all inventions
denom = 0 # 0 = count only inventions on which patents are applied for in denominator of TFPI and RPA, 1 = count all inventions



class Field:

    def __init__(self, a, c, mu, sigma, r, gr, lr, wr, rpa, tfpi):
        """
        Set up field specific parameters.  All parameters are scalars.
        """
        self.a, self.c, self.mu, self.sigma, self.r, self.gr, self.lr, self.wr, self.rpa, self.tfpi = a, c, mu, sigma, r, gr, lr, wr, rpa, tfpi
        if a/2 < c:
            raise ValueError('Assumption on non-drastic innovation violated')

        # a: demand intercept
        # c: unit cost
        # mu: mean of the log of empirically observed patent values, using estimates for log-normal distribution from Bessen (2008)
        # sigma: standard deviation of the log of empirically observed patent values, using estimates for log-normal distribution from Bessen (2008)
        # r: discount rate, set at level of observed depreciation in Bessen (2008)
        # gr: observed grant rate
        # lr: observed litigation rate
        # wr: observed validation rate (patentee win rate in validity challenges)
        # rpa: observed R&D expenditure (adjusted for patent propensity) per patent application
        # tfpi: observed TFP growth per invention

# define various functions

    def pi(self,s,t):
        "profit without patent protection"
        # Unpack parameters (get rid of self to simplify notation)
        a, c, r = self.a, self.c, self.r
        return ((1-np.exp(-r*t))*(a-(1-2*s)*c)**2 + np.exp(-r*t)*(a-(1-s)*c)**2 - (a-c)**2)/(9*r)

    def DeltaH(self,s,t):
        "extra profit from patent when high-type license contract is accepted"
        # Unpack parameters (get rid of self to simplify notation)
        a, c, r = self.a, self.c, self.r
        return ((1-np.exp(-r*T))*5*s*c*(a-c)-(1-np.exp(-r*t))*s*c*(2*(a-c)+3*s*c))/(9*r)

    def LC(self,s,t):
        "litigation cost"
        return l0+l1*self.DeltaH(s,t)

    def DeltaL(self,s,t):
        "low type's preemption payoff"
        # Unpack parameters (get rid of self to simplify notation)
        a, c, r = self.a, self.c, self.r
        if neg_fee == 1:
            return self.LC(s,t)+((1-np.exp(-r*T))*s*c*(a-c)*(1+4*qL)-(1-np.exp(-r*t))*s*c*(2*(a-c)+3*s*c))/(9*r)
        elif neg_fee == 0:
            return (5/4)*self.LC(s,t) + qL*self.DeltaH(s,t) # not adjusted for lead time

    def G(self,kappa,s,k):
        "CDF of R&D costs"
        def Lambda(s):
            "mean R&D cost"
            if negk0==0:
                return k[0]+k[1]*s+k[2]*s**2
            elif negk0==1:
                return k[0]*(1-np.exp(k[1]*np.log(1-s)/((k[0]**2)**(1/2))))+k[1]*s
        return 1-np.exp(-kappa/Lambda(s))

    def x(self,s,t):
        "rate of challenges when high-type license contract is offered" #($\tilde{x}$)
        return (self.DeltaH(s,t)-self.DeltaL(s,t))/((1-qL)*self.DeltaH(s,t)+self.LC(s,t))

    def s_hat(self,e,t):
        "cutoff above which low types apply for patents"
        # Unpack parameters (get rid of self to simplify notation)
        a, c, r = self.a, self.c, self.r
        B = c*(a-c)*(5*(1-np.exp(-r*T))-2*(1-np.exp(-r*t)))
        if t == 0:
            return 9*r*(phiP+phiA/(1-e))/((1-np.exp(-r*T))*5*c*(a-c))
        else:
            return (B-np.sqrt(B**2-108*(1-np.exp(-r*t))*c**2*r*(phiP+phiA/(1-e))))/(6*(1-np.exp(-r*t))*c**2)

    def s_median(self,t):
        "Value of s for which the net-of-fees patent premium, DeltaH-phiA-phiP, is equal to the median of the observed distribution of patent values"
        # Unpack parameters (get rid of self to simplify notation)
        a, c, r = self.a, self.c, self.r
        B = c*(a-c)*(5*(1-np.exp(-r*T))-2*(1-np.exp(-r*t)))
        if t == 0:
            return 9*r*(np.exp(self.mu)+phiP+phiA)/((1-np.exp(-r*T))*5*c*(a-c))
        else:
            return (B-np.sqrt(B**2-108*(1-np.exp(-r*t))*c**2*r*(np.exp(self.mu)+phiP+phiA)))/(6*(1-np.exp(-r*t))*c**2)

    def PiH(self,s,t):
        "high type's expected payoff (gross of R&D cost) when challenges are credible"
        return self.pi(s,t)+(1-self.x(s,t)*(1-qH))*self.DeltaH(s,t)-self.x(s,t)*self.LC(s,t)-phiA-phiP

    def lambdalow(self,s,e,k,t):
        "lowest belief about inventor's type that competitor can hold" #($\underline{\lambda}$)
        return (self.G(self.pi(s,t)+(1-max(0,self.x(s,t))*(1-qH))*self.DeltaH(s,t)-max(0,self.x(s,t))*self.LC(s,t)-phiA-phiP,s,k)-self.G(self.pi(s,t),s,k))/(self.G(self.pi(s,t)+(1-max(0,self.x(s,t))*(1-qH))*self.DeltaH(s,t)-max(0,self.x(s,t))*self.LC(s,t)-phiA-phiP,s,k)-e*self.G(self.pi(s,t),s,k))

    def s_cc(self,e,k,t):
        "cutoff on s above which challenges are credible" #($s^{cc}$)
        def cc(s):
            return (1-qL-self.lambdalow(s,e,k,t)*(qH-qL))*(4/5)*self.DeltaH(s,t)-self.LC(s,t)
        root = optimize.root(cc,x0=0.99)
        return root.x[0] if root.success == True else 1 #'Error: root finding algorithm did not succeed'

    def y(self,s,e,k,t):
        "low type's equilibrium probability of offering the high-type contract when challenges are credible"
        return (((4/5)*self.DeltaH(s,t)*(qH-qL))/((1-qL)*(4/5)*self.DeltaH(s,t)-self.LC(s,t))-1)*(self.G(self.PiH(s,t),s,k)-self.G(self.pi(s,t),s,k))/((1-e)*self.G(self.pi(s,t),s,k))

    def s_bar(self,Beta):
        return (1 - Beta[0]*self.s_median(0))/Beta[1] + self.s_median(0)

    def F(self,s,Beta):
        "CDF of percentage cost reduction s"
        if dist == 0:
            return np.exp(-(s/Beta[0])**(-Beta[1])) # Fréchet with Beta[0] = s (scale), Beta[1]=alpha (shape), and m=0 (minimum)
        elif dist == 1:
            return (s**Beta[0])/(Beta[1]**Beta[0] + s**Beta[0]) # Log-logistic with Beta[0]=beta, Beta[1]=alpha

    def f(self,s,Beta):
        "PDF of percentage cost reduction s"
        if dist == 0:
            return (Beta[1]/Beta[0])*(s/Beta[0])**(-Beta[1]-1)*np.exp(-(s/Beta[0])**(-Beta[1])) # Fréchet with Beta[0] = s (scale), Beta[1]=alpha (shape), and m=0 (minimum)
        elif dist == 1:
            return ((Beta[0]/Beta[1])*(s/Beta[1])**(Beta[0]-1))/(1+(s/Beta[1])**Beta[0])**2 # Log-logistic with Beta[0]=beta, Beta[1]=alpha

    def s0_num(self,t):
        def func(s):
            return self.DeltaH(s,t)-self.LC(s,t)
        root = optimize.root(func,x0=1)
        return root.x[0]

    def s0(self,t):
        "value of s at which DeltaH equals litigation cost"
        # Unpack parameters (get rid of self to simplify notation)
        a, c, r = self.a, self.c, self.r
        B = c*(a-c)*(5*(1-np.exp(-r*T))-2*(1-np.exp(-r*t)))
        if t == 0:
            return 9*r*(l0/(1-l1))/((1-np.exp(-r*T))*5*c*(a-c))
        else:
            return (B-np.sqrt(B**2-108*(1-np.exp(-r*t))*c**2*r*(l0/(1-l1))))/(6*(1-np.exp(-r*t))*c**2)

    def pat_val_dist(self,v):
        "Empirically observed distribution of patent values (from Bessen)"
        V = lognorm(self.sigma,0,np.exp(self.mu))
        return V.cdf(v)

    def s_percentile_v(self,p,t):
        "value of s at which DeltaH-phiA-phiP equals percentile p of the empirically observed distribution of patent values"
        def func(s):
            return self.pat_val_dist(self.DeltaH(s,t)-phiA-phiP)-p
        root = optimize.root(func,x0=self.s_hat(0,t))
        return root.x[0]

    def int_pts(self,e,Beta,t):
        "Special integration points"
        def percentile(p,Beta):
            if dist == 0:
                return Beta[0]*(-np.log(p))**(-1/Beta[1])
            elif dist == 1:
                return Beta[1]*(p/(1-p))**(1/Beta[0])
        smin = percentile(0.01,Beta)
        smax = percentile(0.99,Beta)
        return [smin,smax]

    def AL_nc(self,e,k,Beta,t):
        "Number of applications by low types below s_cc (in per-idea terms)"
        int = lambda s: self.G(self.pi(s,t),s,k)*self.f(s,Beta)
        integral, error = quad(int, self.s_hat(e,t), self.s_cc(e,k,t), points=self.int_pts(e,Beta,t))
        return integral

    def AL_cc(self,e,k,Beta,t):
        "Number of applications by low types above s_cc (in per-idea terms)"
        int = lambda s: self.G(self.pi(s,t),s,k)*self.f(s,Beta)
        integral, error = quad(int, self.s_cc(e,k,t), 1, points=self.int_pts(e,Beta,t))
        return integral

    def AL(self,e,k,Beta,t):
        "Number of applications by low types (in per-idea terms)"
        return self.AL_nc(e,k,Beta,t) + self.AL_cc(e,k,Beta,t)

    def AH_nc(self,e,k,Beta,t):
        "Number of applications by high types below s_cc (in per-idea terms)"
        int_nocc = lambda s: (self.G(self.pi(s,t)+self.DeltaH(s,t)-phiA-phiP,s,k)-self.G(self.pi(s,t),s,k))*self.f(s,Beta)
        integral_nocc, error_nocc = quad(int_nocc, self.s_hat(0,t), self.s_cc(e,k,t), points=self.int_pts(e,Beta,t))
        return integral_nocc

    def AH_cc(self,e,k,Beta,t):
        "Number of applications by high types above s_cc (in per-idea terms)"
        int_cc = lambda s: (self.G(self.PiH(s,t),s,k)-self.G(self.pi(s,t),s,k))*self.f(s,Beta)
        integral_cc, error_cc = quad(int_cc, self.s_cc(e,k,t), 1, points=self.int_pts(e,Beta,t))
        return integral_cc

    def AH(self,e,k,Beta,t):
        "Number of applications by low types (in per-idea terms)"
        return self.AH_nc(e,k,Beta,t) + self.AH_cc(e,k,Beta,t)

    def P_cc(self,e,k,Beta,t):
        "Number of patents for which challenges are credible (in per-idea terms)"
        int = lambda s: (self.G(self.PiH(s,t),s,k)-e*self.G(self.pi(s,t),s,k))*self.f(s,Beta)
        integral, error = quad(int, self.s_cc(e,k,t),1, points=self.int_pts(e,Beta,t))
        return integral

    def P_z(self,z,e,k,Beta,t):
        "Number of patents below and above s=z (in per-idea terms), for z between s_hat and s_cc"
        int0 = lambda s: (self.G(self.pi(s,t)+self.DeltaH(s,t)-phiA-phiP,s,k)-self.G(self.pi(s,t),s,k))*self.f(s,Beta)
        int = lambda s: (self.G(self.pi(s,t)+self.DeltaH(s,t)-phiA-phiP,s,k)-e*self.G(self.pi(s,t),s,k))*self.f(s,Beta)
        integral_0, error_0 = quad(int0, self.s_hat(0,t), self.s_hat(e,t), points=self.int_pts(e,Beta,t))
        integral_u, error_u = quad(int, self.s_hat(e,t),z, points=self.int_pts(e,Beta,t))
        integral_o, error_o = quad(int, z, 1, points=self.int_pts(e,Beta,t))
        return np.array([integral_0+integral_u, integral_o])

    def GR(self,e,k,Beta,t):
        "Grant rate"
        return 1-e*self.AL(e,k,Beta,t)/(self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))

    def X(self,e,k,Beta,t):
        "Number of litigated patents (in per-idea terms)"
        int = lambda s: self.x(s,t)*(self.G(self.PiH(s,t),s,k)-self.G(self.pi(s,t),s,k)+(1-e)*self.y(s,e,k,t)*self.G(self.pi(s,t),s,k))*self.f(s,Beta)
        integral, error = quad(int, self.s_cc(e,k,t), 1, points=self.int_pts(e,Beta,t))
        return integral

    def LR(self,e,k,Beta,t):
        "Litigation rate"
        return self.X(e,k,Beta,t)/(self.AH(e,k,Beta,t)+(1-e)*self.AL(e,k,Beta,t))

    def LR_cc(self,e,k,Beta,t):
        return self.X(e,k,Beta,t)/self.P_cc(e,k,Beta,t)

    def PW(self,e,k,Beta,t):
        "Number of validity challenges won by the patentee (in per-idea terms)"
        int = lambda s: self.x(s,t)*(qH*(self.G(self.PiH(s,t),s,k)-self.G(self.pi(s,t),s,k))+qL*(1-e)*self.y(s,e,k,t)*self.G(self.pi(s,t),s,k))*self.f(s,Beta)
        integral, error = quad(int, self.s_cc(e,k,t), 1, points=self.int_pts(e,Beta,t))
        return integral

    def WR(self,e,k,Beta,t):
        "Validation rate"
        return self.PW(e,k,Beta,t)/self.X(e,k,Beta,t)

    def K(self,e,k,Beta,t):
        "R&D cost for inventions on which a patent application is filed (in per-idea terms)"
        def Lambda(s):
            if negk0==0:
                return k[0]+k[1]*s+k[2]*s**2
            elif negk0==1:
                return k[0]*(1-np.exp(k[1]*np.log(1-s)/((k[0]**2)**(1/2))))+k[1]*s
        def kbar(s,A,B):
            "Expected R&D cost for a given s when kappa is between A and B"
            return -(Lambda(s)+B)*np.exp(-B/Lambda(s)) + (Lambda(s)+A)*np.exp(-A/Lambda(s))
        int_L = lambda s: kbar(s,0,self.pi(s,t))*self.f(s,Beta)
        int_Hnocc = lambda s: kbar(s,self.pi(s,t),self.pi(s,t)+self.DeltaH(s,t)-phiA-phiP)*self.f(s,Beta)
        int_Hcc = lambda s: kbar(s,self.pi(s,t),self.PiH(s,t))*self.f(s,Beta)
        integral_L0, error_L = quad(int_L, self.s_hat(e,t), 1, points=self.int_pts(e,Beta,t))
        integral_L1, error_L = quad(int_L, 0, 1, points=self.int_pts(e,Beta,t))
        integral_Hnocc, error_Hnocc = quad(int_Hnocc, self.s_hat(0,t), self.s_cc(e,k,t), points=self.int_pts(e,Beta,t))
        integral_Hcc, error_Hcc = quad(int_Hcc, self.s_cc(e,k,t), 1, points=self.int_pts(e,Beta,t))
        if num==0:
            return integral_L0 + integral_Hnocc + integral_Hcc
        elif num==1:
            return integral_L1 + integral_Hnocc + integral_Hcc

    def RPA(self,e,k,Beta,t):
        "R&D per patent application"
        if denom==0:
            return self.K(e,k,Beta,t)/(self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))
        elif denom==1:
            return self.K(e,k,Beta,t)/self.NI(e,k,Beta,t)

    def TFP(self,e,k,Beta,t):
        "TFP growth (in per-idea terms)"
        int_L = lambda s: s*self.G(self.pi(s,t),s,k)*self.f(s,Beta)
        int_LHnocc = lambda s: s*self.G(self.pi(s,t)+self.DeltaH(s,t)-phiA-phiP,s,k)*self.f(s,Beta)
        int_LHcc = lambda s: s*self.G(self.PiH(s,t),s,k)*self.f(s,Beta)
        integral_L, error_L = quad(int_L, 0, self.s_hat(0,t), points=self.int_pts(e,Beta,t))
        integral_LHnocc, error_LHnocc = quad(int_LHnocc, self.s_hat(0,t), self.s_cc(e,k,t), points=self.int_pts(e,Beta,t))
        integral_LHcc, error_LHcc = quad(int_LHcc, self.s_cc(e,k,t), 1, points=self.int_pts(e,Beta,t))
        if num==0:
            return integral_LHnocc + integral_LHcc
        elif num==1:
            return integral_L + integral_LHnocc + integral_LHcc

    def NI(self,e,k,Beta,t):
        "Number of inventions (in per-idea terms)"
        int = lambda s: self.G(self.pi(s,t),s,k)*self.f(s,Beta)
        integral, error = quad(int,0,self.s_hat(e,t), points=self.int_pts(e,Beta,t)) # inventions on which no patent application is filed
        return integral + self.AL(e,k,Beta,t) + self.AH(e,k,Beta,t)

    def TFPI(self,e,k,Beta,t):
        "TFP growth per invention"
        if denom==0:
            return self.TFP(e,k,Beta,t)/(self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))
        elif denom==1:
            return self.TFP(e,k,Beta,t)/self.NI(e,k,Beta,t)

    def SP_cc(self,e,k,Beta,t):
        "Share of patents for which challenges are credible"
        return self.P_cc(e,k,Beta,t)/((1-e)*self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))

    def SP_z(self,z,e,k,Beta,t):
        "Share of patents below and above s=z (for z between s_hat(e,t) and s_cc)"
        return self.P_z(z,e,k,Beta,t)/((1-e)*self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))

    def eqs(self,x):
        e = x[0]
        k = [x[1],x[2]]
        Beta = [x[3],x[4]]
        t = x[5]
        return np.array([self.GR(e,k,Beta,t)-self.gr,self.LR(e,k,Beta,t)-self.lr,self.WR(e,k,Beta,t)-self.wr,self.RPA(e,k,Beta,t)-self.rpa,self.TFPI(e,k,Beta,t)-self.tfpi])

    def eqs_nd(self,x): # no division
        e = x[0]
        k = [x[1],x[2],0]
        Beta = [x[3],x[4]]
        t = 0
        if denom==0:
            return np.array([(self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))*self.gr-((1-e)*self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t)), ((1-e)*self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))*self.lr-self.X(e,k,Beta,t), self.X(e,k,Beta,t)*self.wr-self.PW(e,k,Beta,t), (self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))*self.rpa-self.K(e,k,Beta,t), (self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))*self.tfpi-self.TFP(e,k,Beta,t)])
        elif denom==1:
            return np.array([(self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))*self.gr-((1-e)*self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t)), ((1-e)*self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))*self.lr-self.X(e,k,Beta,t), self.X(e,k,Beta,t)*self.wr-self.PW(e,k,Beta,t), self.NI(e,k,Beta,t)*self.rpa-self.K(e,k,Beta,t), self.NI(e,k,Beta,t)*self.tfpi-self.TFP(e,k,Beta,t)])
        #self.SP_z(self.s_median(t),e,k,Beta,t)[0]-0.5])  #self.SP_z(s0,e,k,Beta)[0] - self.pat_val_dist(self.DeltaH(s0)), self.SP_z(self.s_percentile_v(0.5),e,k,Beta)[0] - 0.5])

    def eqs_rel(self,x,w_gr,w_lr,w_wr,w_rpa,w_tfpi): #,w_spm):
        e = x[0]
        k = [x[1], x[2], 0]
        Beta = [x[3],x[4]]
        t = 0
        return np.array([w_gr*(self.GR(e,k,Beta,t)-self.gr)/self.gr,w_lr*(self.LR(e,k,Beta,t)-self.lr)/self.lr,w_wr*(self.WR(e,k,Beta,t)-self.wr)/self.wr,w_rpa*(self.RPA(e,k,Beta,t)-self.rpa)/self.rpa,w_tfpi*(self.TFPI(e,k,Beta,t)-self.tfpi)/self.tfpi]) #,(self.SP_z(self.s_median(t),e,k,Beta,t)[0]-0.5)/0.5])

    def estimation(self,x0):
        solution = optimize.root(self.eqs_nd,x0,method='hybr')
        return solution.x

    def startval(self):
        x0 = [0.5,50000,0,0]
        factor_k1 = (self.a-self.c)*self.tfpi*self.c
        factor_beta = (self.a-self.c)*self.tfpi*self.c/self.rpa
        x0[2] = 12000*factor_k1
        x0[3] = 3500*factor_beta
        return np.array(x0)

    def estimation_lq(self,x0,weights):
        if negk0==0:
            lowerbounds = [0,0,0,0,0]
        if negk0==1:
            lowerbounds = [0,-np.inf,0,0,0]
        upperbounds = [1,np.inf,np.inf,np.inf,1]
        solution = optimize.least_squares(self.eqs_rel,x0,args=weights,bounds=(lowerbounds,upperbounds),verbose=2) #ftol=0.001,
        return solution

    def sumsquares(self,x):
        squares = self.eqs_rel(x,1,1,1,1,1,1)**2
        sumsquares = squares.sum()
        return sumsquares

    def estimation_shgo(self):
        bounds = [(0,ebar),(0,None),(0,None),(0,None),(1,100000)]
        solution = optimize.shgo(self.sumsquares,bounds)
        return solution

#    def estimation_brute(self,n):
#        factor_k1 = (self.a-self.c)*self.tfp*self.c
#        factor_beta = (self.a-self.c)*self.tfp*self.c/self.rpa
#        k1_low, k1_high = 10000*factor_k1, 14000*factor_k1
#        beta_low, beta_high = 2000*factor_beta, 5000*factor_beta
#        rranges = (slice(0.2, 0.6, 0.4/n), slice(0, 100000, 100000/n), slice(k1_low, k1_high,(k1_high-k1_low)/n), slice(beta_low, beta_high, (beta_high-beta_low)/n))
#        resbrute = optimize.brute(self.sumsquares, rranges, full_output=True, finish=None)
#        return resbrute

    def totals(self,x):
        e,k,Beta,t = x
        return self.AL(e,k,Beta,t), self.AH(e,k,Beta,t), self.X(e,k,Beta,t), self.PW(e,k,Beta,t), self.K(e,k,Beta,t), self.TFP(e,k,Beta,t), self.NI(e,k,Beta,t)

    def targets(self,x):
        e,k,Beta,t = x
        return self.GR(e,k,Beta,t), self.LR(e,k,Beta,t), self.WR(e,k,Beta,t), self.RPA(e,k,Beta,t), self.TFPI(e,k,Beta,t), self.SP_z(self.s_median(t),e,k,Beta,t)[0] #, self.SP_cc(e,k,Beta,t), (1-self.pat_val_dist(self.EVscc(e,k)))

    def SHA(self,x):
        "Share of high types among applicants"
        e,k,Beta,t = x
        return self.AH(e,k,Beta,t)/(self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))

    def SHA_nc(self,x):
        "Share of high types among applicants below s_cc"
        e,k,Beta,t = x
        return self.AH_nc(e,k,Beta,t)/(self.AL_nc(e,k,Beta,t)+self.AH_nc(e,k,Beta,t))

    def SHA_cc(self,x):
        "Share of high types among applicants above s_cc"
        e,k,Beta,t = x
        return self.AH_cc(e,k,Beta,t)/(self.AL_cc(e,k,Beta,t)+self.AH_cc(e,k,Beta,t))

    def SHP(self,x):
        "Share of high types among patentees"
        e,k,Beta,t = x
        return self.AH(e,k,Beta,t)/((1-e)*self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))

    def SHP_nc(self,x):
        "Share of high types among patentees below s_cc"
        e,k,Beta,t = x
        return self.AH_nc(e,k,Beta,t)/((1-e)*self.AL_nc(e,k,Beta,t)+self.AH_nc(e,k,Beta,t))

    def SHP_cc(self,x):
        "Share of high types among patentees above s_cc"
        e,k,Beta,t = x
        return self.AH_cc(e,k,Beta,t)/((1-e)*self.AL_cc(e,k,Beta,t)+self.AH_cc(e,k,Beta,t))

    def xbar(self,x):
        "mean rate of challenges conditional on s being above scc"
        e,k,Beta,t = x
        int = lambda s: self.x(s,t)*self.f(s,Beta)
        integral, error = quad(int, self.s_cc(e,k,t),1, points=self.int_pts(e,Beta,t))
        return integral/(1-self.F(self.s_cc(e,k,t),Beta))

    def ybar(self,x):
        "mean rate of offering high-type license contract conditional on s being above scc"
        e,k,Beta,t = x
        int = lambda s: self.y(s,e,k,t)*self.f(s,Beta)
        integral, error = quad(int, self.s_cc(e,k,t),1, points=self.int_pts(e,Beta,t))
        return integral/(1-self.F(self.s_cc(e,k,t),Beta))

    def FR(self,e,k,Beta,t):
        "Fee revenue collected by the patent office"
        return (phiA-15000)*(self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t)) + phiP*((1-e)*self.AL(e,k,Beta,t)+self.AH(e,k,Beta,t))

#    def gamma1(self):
#        "Exponent of the examination cost function"
#        return -eta*self.gr/(1-self.gr)
    def g(self,e):
        "Function g(e) used in calibration of examination cost"
        return e/(1-e)

    def gamma(self,gamma0,gamma1,e):
        "Examination cost function (per application)"
        return (gamma0+self.g(e))**gamma1

    def percent_residuals(self,x,y):
        e = x[0]
        k = [x[1],x[2],0]
        Beta = [x[3],x[4]]
        t = 0
        predict = [self.GR(e,k,Beta,t), self.LR(e,k,Beta,t), self.WR(e,k,Beta,t), self.RPA(e,k,Beta,t), self.TFPI(e,k,Beta,t)]
        percent = []
        residual = []
        for i in range(len(y)):
            residual.append(y[i] - predict[i])
            percent.append(residual[i]/y[i])
        return percent

    def residuals(self,x,y):
        e = x[0]
        k = [x[1],x[2],0]
        Beta = [x[3],x[4]]
        t = 0
        predict = [self.GR(e,k,Beta,t), self.LR(e,k,Beta,t), self.WR(e,k,Beta,t), self.RPA(e,k,Beta,t), self.TFPI(e,k,Beta,t)]
        percent = []
        residual = []
        for i in range(len(y)):
            residual.append(y[i] - predict[i])
            percent.append(residual[i]/y[i])
        return residual
    
    def test_residuals(self,x,y):
        e = x[0]
        k = [x[1],x[2],0]
        Beta = [x[3],x[4]]
        t = 0
        predict = [self.GR(e,k,Beta,t), self.LR(e,k,Beta,t), self.WR(e,k,Beta,t), (1e-6)*self.RPA(e,k,Beta,t), (1e4)*self.TFPI(e,k,Beta,t)]
        percent = []
        residual = []
        for i in range(len(y)):
            residual.append(y[i] - predict[i])           
        return residual

    def weighted_sumsq(self, x, y):
        e = x[0]
        k = [x[1],x[2],0]
        Beta = [x[3],x[4]]
        t = 0
        predict = [self.GR(e,k,Beta,t), self.LR(e,k,Beta,t), self.WR(e,k,Beta,t), self.RPA(e,k,Beta,t), self.TFPI(e,k,Beta,t)]
        percent = []
        residual = []
        for i in range(len(y)):
            residual.append(y[i] - predict[i])
            percent.append(residual[i]/y[i])
        s = np.dot(residual, [1, 100, 1, 1e-7, 1e5])
        return s

      
# all below needs to be adjusted for lead time

    def RoyaltyH(self,s):
        "royalty charged by the high type"
        return s * self.c

    def RoyaltyL(self,s,t):
        "royalty charged by the low type (with prob. 1-y)"
        Psi = (self.a-(1-s)*self.c)**2 - 9*self.r*self.LC(s,t)/(1-np.exp(-self.r*T))-4*qL*(self.a-self.c)*s*self.c # not adjusted for lead time
        if neg_fee == 1:
            return s * self.c
        elif neg_fee == 0:
            return (1/2)*(self.a-(1-s)*self.c-Psi**(1/2)) # not adjusted for lead time

    def w(self,s,t,cs):
        "welfare gain from invention with cost reduction s in the absence of patent protection"
        if cs == 0:
            return 4*self.pi(s,t)
        elif cs == 1:
            return 2*self.pi(s,t)

    def DWL(self,s,R,cs):
        "loss in total or consumer surplus due to patent protection for invention with cost reduction s"
        if cs == 0:
            return (1-np.exp(-self.r*T))*(self.a-(1-s)*self.c+R/2)*R/(9*self.r) # not adjusted for lead time
        elif cs == 1:
            return (1-np.exp(-self.r*T))*(2*(self.a-(1-s)*self.c)-R/2)*R/(9*self.r) # not adjusted for lead time

    def Surplus(self,phiA,phiP,e,k,Beta,t,cs):
        "total or consumer surplus gain from innovation in the tech field (gross of examination costs)"
        def shat(phiA,phiP,e):
            return 9*self.r*(phiP+phiA/(1-e))/((1-np.exp(-self.r*T))*5*self.c*(self.a-self.c)) # not adjusted for lead time
        def PiH(phiA,phiP,s,t):
            return self.pi(s,t)+(1-self.x(s,t)*(1-qH))*self.DeltaH(s,t)-self.x(s,t)*self.LC(s,t)-phiA-phiP
        def lambdalow(phiA,phiP,s,e,k,t):
            return (self.G(self.pi(s,t)+(1-max(0,self.x(s,t))*(1-qH))*self.DeltaH(s,t)-max(0,self.x(s,t))*self.LC(s,t)-phiA-phiP,s,k)-self.G(self.pi(s,t),s,k))/(self.G(self.pi(s,t)+(1-max(0,self.x(s,t))*(1-qH))*self.DeltaH(s,t)-max(0,self.x(s,t))*self.LC(s,t)-phiA-phiP,s,k)-e*self.G(self.pi(s,t),s,k))
        def scc(phiA,phiP,e,k,t):
            def cc(s):
                return (1-qL-lambdalow(phiA,phiP,s,e,k,t)*(qH-qL))*(4/5)*self.DeltaH(s,t)-self.LC(s,t)
            root = optimize.root(cc,x0=1)
            return root.x[0] if root.success == True else 1 #'Error: root finding algorithm did not succeed'
        def y(phiA,phiP,s,e,k,t):
            return (((4/5)*self.DeltaH(s,t)*(qH-qL))/((1-qL)*(4/5)*self.DeltaH(s,t)-self.LC(s,t))-1)*(self.G(PiH(phiA,phiP,s,t),s,k)-self.G(self.pi(s,t),s,k))/((1-e)*self.G(self.pi(s,t),s,k))
        def Lambda(s):
            if negk0==0:
                return k[0]+k[1]*s+k[2]*s**2
            elif negk0==1:
                return k[0]*(1-np.exp(k[1]*np.log(1-s)/((k[0]**2)**(1/2))))+k[1]*s
        def kbar(s,A,B):
            "Expected R&D cost for a given s when kappa is between A and B"
            return -(Lambda(s)+B)*np.exp(-B/Lambda(s)) + (Lambda(s)+A)*np.exp(-A/Lambda(s))
        int_L = lambda s: kbar(s,0,self.pi(s,t))*self.f(s,Beta)
        int_Hnocc = lambda s: kbar(s,self.pi(s,t),self.pi(s,t)+self.DeltaH(s,t)-phiA-phiP)*self.f(s,Beta)
        int_Hcc = lambda s: kbar(s,self.pi(s,t),PiH(phiA,phiP,s,t))*self.f(s,Beta)
        integral_L, error_L = quad(int_L, 0, 1, points=self.int_pts(0,Beta,t))
        integral_Hnocc, error_Hnocc = quad(int_Hnocc, shat(phiA,phiP,0), scc(phiA,phiP,e,k,t), points=self.int_pts(0,Beta,t))
        integral_Hcc, error_Hcc = quad(int_Hcc, scc(phiA,phiP,e,k,t), 1, points=self.int_pts(0,Beta,t))
        int_1 = lambda s: self.G(self.pi(s,t),s,k)*self.w(s,t,cs)*self.f(s,Beta)
        int_2 = lambda s: self.G(self.pi(s,t),s,k)*(self.w(s,t,cs)-(1-e)*self.DWL(s,self.RoyaltyH(s),cs))*self.f(s,Beta)
        int_3 = lambda s: (self.G(self.pi(s,t)+self.DeltaH(s,t)-phiA-phiP,s,k)-self.G(self.pi(s,t),s,k))*(self.w(s,t,cs)-self.DWL(s,self.RoyaltyH(s),cs))*self.f(s,Beta)
        int_4 = lambda s: (self.G(self.pi(s,t),s,k)*(self.w(s,t,cs)-(1-e)*(y(phiA,phiP,s,e,k,t)*(self.x(s,t)*(qL*self.DWL(s,self.RoyaltyH(s),cs)+2*self.LC(s,t))+(1-self.x(s,t))*self.DWL(s,self.RoyaltyH(s),cs))+(1-y(phiA,phiP,s,e,k,t))*self.DWL(s,self.RoyaltyL(s,t),cs)))+(self.G(self.pi(s,t)+self.DeltaH(s,t)-self.x(s,t)*self.LC(s,t)-phiA-phiP,s,k)-self.G(self.pi(s,t),s,k))*(self.w(s,t,cs)-self.x(s,t)*(qH*self.DWL(s,self.RoyaltyH(s),cs)+2*self.LC(s,t))-(1-self.x(s,t))*self.DWL(s,self.RoyaltyH(s),cs)))*self.f(s,Beta)
        integral_1, error_1 = quad(int_1, 0, shat(phiA,phiP,e), points=self.int_pts(0,Beta,t))
        integral_2, error_2 = quad(int_2, shat(phiA,phiP,e), scc(phiA,phiP,e,k,t), points=self.int_pts(0,Beta,t))
        integral_3, error_3 = quad(int_3, shat(phiA,phiP,0), scc(phiA,phiP,e,k,t), points=self.int_pts(0,Beta,t))
        integral_4, error_4 = quad(int_4, scc(phiA,phiP,e,k,t), 1, points=self.int_pts(0,Beta,t))
        return integral_1 + integral_2 + integral_3 + integral_4 - (integral_L + integral_Hnocc + integral_Hcc)

    def W(self,phiA,phiP,e,k,Beta,t):
        "total surplus gain from innovation in the tech field (gross of examination costs)"
        return self.Surplus(phiA,phiP,e,k,Beta,t,0)

    def CS(self,phiA,phiP,e,k,Beta,t):
        "consumer surplus gain from innovation in the tech field (gross of examination costs)"
        return self.Surplus(phiA,phiP,e,k,Beta,t,1)

    def Gamma(self,phiA,phiP,e,k,Beta,t,gamma0,gamma1):
        "Total cost of patent examination"
        def shat(phiA,phiP,e):
            return 9*self.r*(phiP+phiA/(1-e))/((1-np.exp(-self.r*T))*5*self.c*(self.a-self.c)) # not adjusted for lead time
        def PiH(phiA,phiP,s,t):
            return self.pi(s,t)+(1-self.x(s,t)*(1-qH))*self.DeltaH(s,t)-self.x(s,t)*self.LC(s,t)-phiA-phiP
        def lambdalow(phiA,phiP,s,e,k,t):
            return (self.G(self.pi(s,t)+(1-max(0,self.x(s,t))*(1-qH))*self.DeltaH(s,t)-max(0,self.x(s,t))*self.LC(s,t)-phiA-phiP,s,k)-self.G(self.pi(s,t),s,k))/(self.G(self.pi(s,t)+(1-max(0,self.x(s,t))*(1-qH))*self.DeltaH(s,t)-max(0,self.x(s,t))*self.LC(s,t)-phiA-phiP,s,k)-e*self.G(self.pi(s,t),s,k))
        def scc(phiA,phiP,e,k,t):
            def cc(s):
                return (1-qL-lambdalow(phiA,phiP,s,e,k,t)*(qH-qL))*(4/5)*self.DeltaH(s,t)-self.LC(s,t)
            root = optimize.root(cc,x0=1)
            return root.x[0] if root.success == True else 1 #'Error: root finding algorithm did not succeed'
        def A(phiA,phiP,e):
            int_L = lambda s: self.G(self.pi(s,t),s,k)*self.f(s,Beta)
            int_nocc = lambda s: (self.G(self.pi(s,t)+self.DeltaH(s,t)-phiA-phiP,s,k)-self.G(self.pi(s,t),s,k))*self.f(s,Beta)
            int_cc = lambda s: (self.G(PiH(phiA,phiP,s,t),s,k)-self.G(self.pi(s,t),s,k))*self.f(s,Beta)
            integral_L, error_L = quad(int_L, shat(phiA,phiP,e), 1, points=self.int_pts(0,Beta,t))
            integral_nocc, error_nocc = quad(int_nocc, shat(phiA,phiP,0), scc(phiA,phiP,e,k,t), points=self.int_pts(0,Beta,t))
            integral_cc, error_cc = quad(int_cc, scc(phiA,phiP,e,k,t), 1, points=self.int_pts(0,Beta,t))
            return integral_L + integral_nocc + integral_cc
        return A(phiA,phiP,e)*self.gamma(gamma0,gamma1,e)

    def q(self, x, prepon=1):
        "Compute q_L and q_H. If prepon=1, use preponderance of evidence, lamda_star=0.5. If prepon=0, use clear and convincing evidence, lamda_star=0.8"
        e,k,Beta,t = x
        lamda_star=0.5 if prepon==1 else 0.8
        lamda = self.SHP(x)
        s_l = (2 + sqrt(1 - e)) / (3 + e)
        s_h = 1 - s_l
        def f_h(s):
            if s >= s_h and s <= 1:
                f_h = 2*(s - s_h) / (1 - s_h)**2
            else:
                f_h = 0
            return f_h
        def f_l(s):
            if s >= 0 and s <= s_l:
                f_l = 2*(s_l - s) / (s_l)**2
            else:
                f_l = 0
            return f_l
        def eq(s):
            return (lamda * f_h(s)) / (lamda * f_h(s) + (1 - lamda) * f_l(s)) - lamda_star
        def F_l(s):
            if s < 0:
                return 0
            elif s > s_l:
                return 1
            else:
                return s * (2 * s_l - s) / (s_l)**2
        def F_h(s):
            if s < s_h:
                return 0
            elif s > 1:
                return 1
            else:
                return (s * (s - 2*s_h) + s_h**2) / (1 - s_h)**2       
        sigma_star = optimize.root(eq, x0=0.5).x[0]
        q_l = 1 - F_l(sigma_star)
        q_h = 1 - F_h(sigma_star)
        return q_l, q_h


class Welfare:

    def __init__(self, k, Beta, t, gamma):
        self.k, self.Beta, self.t, self.gamma = k, Beta, t, gamma

        # k: [k[0], k[1], k[2]] vector of estimated parameters of the distribution of R&D cost G
        # Beta: [Beta[0], Beta[1]] vector of estimated parameters of the distribution of cost reductions F
        # t: lead time for non-patented inventions
        # gamma: [gamma[0], gamma[1]] vector of estimated parameters of the examination cost function

# define auxiliary functions

    def shat(self,field,phiA,phiP,e):
        return 9*field.r*(phiP+phiA/(1-e))/((1-np.exp(-field.r*T))*5*field.c*(field.a-field.c)) # not adjusted for lead time

    def PiH(self,field,phiA,phiP,s):
        t = self.t
        return field.pi(s,t)+(1-field.x(s,t)*(1-qH))*field.DeltaH(s,t)-field.x(s,t)*field.LC(s,t)-phiA-phiP

    def lambdalow(self,field,phiA,phiP,e,s):
        k, t = self.k, self.t
        return (field.G(field.pi(s,t)+(1-max(0,field.x(s,t))*(1-qH))*field.DeltaH(s,t)-max(0,field.x(s,t))*field.LC(s,t)-phiA-phiP,s,k)-field.G(field.pi(s,t),s,k))/(field.G(field.pi(s,t)+(1-max(0,field.x(s,t))*(1-qH))*field.DeltaH(s,t)-max(0,field.x(s,t))*field.LC(s,t)-phiA-phiP,s,k)-e*field.G(field.pi(s,t),s,k))

    def scc(self,field,phiA,phiP,e):
        k, t = self.k, self.t
        def cc(s):
            return (1-qL-self.lambdalow(field,phiA,phiP,e,s)*(qH-qL))*(4/5)*field.DeltaH(s,t)-field.LC(s,t)
        root = optimize.root(cc,x0=1)
        return root.x[0] if root.success == True else 1 #'Error: root finding algorithm did not succeed'

    def y(self,field,phiA,phiP,e,s):
        k, t = self.k, self.t
        return (((4/5)*field.DeltaH(s,t)*(qH-qL))/((1-qL)*(4/5)*field.DeltaH(s,t)-field.LC(s,t))-1)*(field.G(self.PiH(field,phiA,phiP,s),s,k)-field.G(field.pi(s,t),s,k))/((1-e)*field.G(field.pi(s,t),s,k))

    def Lambda(self,s):
        k = self.k
        if negk0==0:
            return k[0]+k[1]*s+k[2]*s**2
        elif negk0==1:
            return k[0]*(1-np.exp(k[1]*np.log(1-s)/((k[0]**2)**(1/2))))+k[1]*s

    def kbar(self,s,A,B):
        "Expected R&D cost for a given s when kappa is between A and B"
        return -(self.Lambda(s)+B)*np.exp(-B/self.Lambda(s)) + (self.Lambda(s)+A)*np.exp(-A/self.Lambda(s))

    def A(self,field,phiA,phiP,e):
        k, Beta, t = self.k, self.Beta, self.t
        int_L = lambda s: field.G(field.pi(s,t),s,k)*field.f(s,Beta)
        int_nocc = lambda s: (field.G(field.pi(s,t)+field.DeltaH(s,t)-phiA-phiP,s,k)-field.G(field.pi(s,t),s,k))*field.f(s,Beta)
        int_cc = lambda s: (field.G(self.PiH(field,phiA,phiP,s),s,k)-field.G(field.pi(s,t),s,k))*field.f(s,Beta)
        integral_L, error_L = quad(int_L, self.shat(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        integral_nocc, error_nocc = quad(int_nocc, self.shat(field,phiA,phiP,0), self.scc(field,phiA,phiP,e), points=field.int_pts(0,Beta,t))
        integral_cc, error_cc = quad(int_cc, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        return integral_L + integral_nocc + integral_cc


# surplus calculation

    def surplus(self,field,phiA,phiP,e,cs):
        "total or consumer surplus gain from innovation in the tech field (gross of examination costs)"
        k, Beta, t = self.k, self.Beta, self.t
        int_L = lambda s: self.kbar(s,0,field.pi(s,t))*field.f(s,Beta)
        int_Hnocc = lambda s: self.kbar(s,field.pi(s,t),field.pi(s,t)+field.DeltaH(s,t)-phiA-phiP)*field.f(s,Beta)
        int_Hcc = lambda s: self.kbar(s,field.pi(s,t),self.PiH(field,phiA,phiP,s))*field.f(s,Beta)
        integral_L, error_L = quad(int_L, 0, 1, points=field.int_pts(0,Beta,t))
        integral_Hnocc, error_Hnocc = quad(int_Hnocc, self.shat(field,phiA,phiP,0), self.scc(field,phiA,phiP,e), points=field.int_pts(0,Beta,t))
        integral_Hcc, error_Hcc = quad(int_Hcc, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        int_1 = lambda s: field.G(field.pi(s,t),s,k)*field.w(s,t,cs)*field.f(s,Beta)
        int_2 = lambda s: field.G(field.pi(s,t),s,k)*(field.w(s,t,cs)-(1-e)*field.DWL(s,field.RoyaltyH(s),cs))*field.f(s,Beta)
        int_3 = lambda s: (field.G(field.pi(s,t)+field.DeltaH(s,t)-phiA-phiP,s,k)-field.G(field.pi(s,t),s,k))*(field.w(s,t,cs)-field.DWL(s,field.RoyaltyH(s),cs))*field.f(s,Beta)
        int_4 = lambda s: (field.G(field.pi(s,t),s,k)*(field.w(s,t,cs)-(1-e)*(self.y(field,phiA,phiP,e,s)*(field.x(s,t)*(qL*field.DWL(s,field.RoyaltyH(s),cs)+2*field.LC(s,t))+(1-field.x(s,t))*field.DWL(s,field.RoyaltyH(s),cs))+(1-self.y(field,phiA,phiP,e,s))*field.DWL(s,field.RoyaltyL(s,t),cs)))+(field.G(field.pi(s,t)+field.DeltaH(s,t)-field.x(s,t)*field.LC(s,t)-phiA-phiP,s,k)-field.G(field.pi(s,t),s,k))*(field.w(s,t,cs)-field.x(s,t)*(qH*field.DWL(s,field.RoyaltyH(s),cs)+2*field.LC(s,t))-(1-field.x(s,t))*field.DWL(s,field.RoyaltyH(s),cs)))*field.f(s,Beta)
        integral_1, error_1 = quad(int_1, 0, self.shat(field,phiA,phiP,e), points=field.int_pts(0,Beta,t))
        integral_2, error_2 = quad(int_2, self.shat(field,phiA,phiP,e), self.scc(field,phiA,phiP,e), points=field.int_pts(0,Beta,t))
        integral_3, error_3 = quad(int_3, self.shat(field,phiA,phiP,0), self.scc(field,phiA,phiP,e), points=field.int_pts(0,Beta,t))
        integral_4, error_4 = quad(int_4, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        return integral_1 + integral_2 + integral_3 + integral_4 - (integral_L + integral_Hnocc + integral_Hcc)

    def W(self,x):
        "Welfare (total surplus) gross of examination costs"
        field,phiA,phiP,e = x
        return self.surplus(field,phiA,phiP,e,0)

    def CS(self,x):
        "Consumer surplus"
        field,phiA,phiP,e = x
        return self.surplus(field,phiA,phiP,e,1)

    def Gamma(self,x):
        "Total cost of patent examination"
        field,phiA,phiP,e = x
        k, Beta, t, gamma = self.k, self.Beta, self.t, self.gamma
        return self.A(field,phiA,phiP,e)*field.gamma(gamma[0],gamma[1],e)


# welfare decomposition

    def I_Lnc(self,x):
        "Low-type innovation when challenges are not credible"
        field,phiA,phiP,e = x
        k, Beta, t = self.k, self.Beta, self.t
        cs = 0
        int_k = lambda s: self.kbar(s,0,field.pi(s,t))*field.f(s,Beta)
        int = lambda s: field.G(field.pi(s,t),s,k)*field.w(s,t,cs)*field.f(s,Beta)
        integral_k, error_k = quad(int_k, 0, self.scc(field,phiA,phiP,e), points=field.int_pts(0,Beta,t))
        integral, error = quad(int, 0, self.scc(field,phiA,phiP,e), points=field.int_pts(0,Beta,t))
        return integral - integral_k

    def I_Lcc(self,x):
        "Low-type innovation when challenges are credible"
        field,phiA,phiP,e = x
        k, Beta, t = self.k, self.Beta, self.t
        cs = 0
        int_k = lambda s: self.kbar(s,0,field.pi(s,t))*field.f(s,Beta)
        int = lambda s: field.G(field.pi(s,t),s,k)*field.w(s,t,cs)*field.f(s,Beta)
        integral_k, error_k = quad(int_k, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        integral, error = quad(int, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        return integral - integral_k

    def I_Hnc(self,x):
        "High-type innovation when challenges are not credible"
        field,phiA,phiP,e = x
        k, Beta, t = self.k, self.Beta, self.t
        cs = 0
        int_k = lambda s: self.kbar(s,field.pi(s,t),field.pi(s,t)+field.DeltaH(s,t)-phiA-phiP)*field.f(s,Beta)
        int = lambda s: (field.G(field.pi(s,t)+field.DeltaH(s,t)-phiA-phiP,s,k)-field.G(field.pi(s,t),s,k))*field.w(s,t,cs)*field.f(s,Beta)
        integral_k, error_k = quad(int_k, self.shat(field,phiA,phiP,0), self.scc(field,phiA,phiP,e), points=field.int_pts(0,Beta,t))
        integral, error = quad(int, self.shat(field,phiA,phiP,0), self.scc(field,phiA,phiP,e), points=field.int_pts(0,Beta,t))
        return integral - integral_k

    def I_Hcc(self,x):
        "High-type innovation when challenges are credible"
        field,phiA,phiP,e = x
        k, Beta, t = self.k, self.Beta, self.t
        cs = 0
        int_k = lambda s: self.kbar(s,field.pi(s,t),self.PiH(field,phiA,phiP,s))*field.f(s,Beta)
        int = lambda s: (field.G(self.PiH(field,phiA,phiP,s),s,k)-field.G(field.pi(s,t),s,k))*field.w(s,t,cs)*field.f(s,Beta)
        integral_k, error_k = quad(int_k, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        integral, error = quad(int, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        return integral - integral_k

    #def intfunc(self,Htype,credible,integrand,x):
    #    field,phiA,phiP,e = x
    #    k, Beta, t = self.k, self.Beta, self.t
    #    cs = 0
    #    if Htype==0 and credible==0:
    #        A,B,C = self.shat(field,phiA,phiP,e), self.scc(field,phiA,phiP,e), field.G(field.pi(s,t),s,k)
    #    elif Htype==0 and credible==1:
    #        A,B,C = self.scc(field,phiA,phiP,e), 1, field.G(field.pi(s,t),s,k)
    #    elif Htype==1 and credible==0:
    #        A,B,C = self.shat(field,phiA,phiP,0), self.scc(field,phiA,phiP,e), (field.G(field.pi(s,t)+field.DeltaH(s,t)-phiA-phiP,s,k)-field.G(field.pi(s,t),s,k))
    #    elif Htype==1 and credible==1:
    #        A,B,C = self.scc(field,phiA,phiP,e), 1, (field.G(self.PiH(field,phiA,phiP,s),s,k)-field.G(field.pi(s,t),s,k))
    #    int = lambda s: C*integrand*field.f(s,Beta)
    #    integral, error = quad(int, A, B, points=field.int_pts(0,Beta,t))
    #    return integral

    #def DWL_Hccalt(self,x):
    #    return self.intfunc(1,1,field.DWL(s,field.RoyaltyH(s),cs),x)

    def DWL_Lnc(self,x):
        "Deadweight loss from low types when challenges are not credible"
        field,phiA,phiP,e = x
        k, Beta, t = self.k, self.Beta, self.t
        cs = 0
        int = lambda s: field.G(field.pi(s,t),s,k)*(1-e)*field.DWL(s,field.RoyaltyH(s),cs)*field.f(s,Beta)
        integral, error = quad(int, self.shat(field,phiA,phiP,e), self.scc(field,phiA,phiP,e), points=field.int_pts(0,Beta,t))
        return integral

    def DWL_Lcc(self,x):
        "Gross deadweight loss (before challenges) from low types when challenges are credible"
        field,phiA,phiP,e = x
        k, Beta, t = self.k, self.Beta, self.t
        cs = 0
        int = lambda s: field.G(field.pi(s,t),s,k)*(1-e)*field.DWL(s,field.RoyaltyH(s),cs)*field.f(s,Beta)
        integral, error = quad(int, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        return integral

    def DWL_Hnc(self,x):
        "Deadweight loss from high types when challenges are not credible"
        field,phiA,phiP,e = x
        k, Beta, t = self.k, self.Beta, self.t
        cs = 0
        int = lambda s: (field.G(field.pi(s,t)+field.DeltaH(s,t)-phiA-phiP,s,k)-field.G(field.pi(s,t),s,k))*field.DWL(s,field.RoyaltyH(s),cs)*field.f(s,Beta)
        integral, error = quad(int, self.shat(field,phiA,phiP,0), self.scc(field,phiA,phiP,e), points=field.int_pts(0,Beta,t))
        return integral

    def DWL_Hcc(self,x):
        "Gross deadweight loss (before challenges) from high types when challenges are credible"
        field,phiA,phiP,e = x
        k, Beta, t = self.k, self.Beta, self.t
        cs = 0
        int = lambda s: (field.G(self.PiH(field,phiA,phiP,s),s,k)-field.G(field.pi(s,t),s,k))*field.DWL(s,field.RoyaltyH(s),cs)*field.f(s,Beta)
        integral, error = quad(int, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        return integral

    def DA_L(self,x):
        "Low-type deadweight loss avoided thanks to challenges"
        field,phiA,phiP,e = x
        k, Beta, t = self.k, self.Beta, self.t
        cs = 0
        int = lambda s: field.G(field.pi(s,t),s,k)*(1-e)*(self.y(field,phiA,phiP,e,s)*field.x(s,t)*(1-qL)*field.DWL(s,field.RoyaltyH(s),cs) + (1-self.y(field,phiA,phiP,e,s))*(field.DWL(s,field.RoyaltyH(s),cs)-field.DWL(s,field.RoyaltyL(s,t),cs)))*field.f(s,Beta)
        integral, error = quad(int, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        return integral

    def DA_H(self,x):
        "High-type deadweight loss avoided thanks to challenges"
        field,phiA,phiP,e = x
        k, Beta, t = self.k, self.Beta, self.t
        cs = 0
        int = lambda s: (field.G(self.PiH(field,phiA,phiP,s),s,k)-field.G(field.pi(s,t),s,k))*field.x(s,t)*(1-qH)*field.DWL(s,field.RoyaltyH(s),cs)*field.f(s,Beta)
        integral, error = quad(int, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        return integral

    def LC_L(self,x):
        "Litigation costs associated with challenges of low-type patents"
        field,phiA,phiP,e = x
        k, Beta, t = self.k, self.Beta, self.t
        cs = 0
        int = lambda s: field.G(field.pi(s,t),s,k)*(1-e)*self.y(field,phiA,phiP,e,s)*field.x(s,t)*2*field.LC(s,t)*field.f(s,Beta)
        integral, error = quad(int, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        return integral

    def LC_H(self,x):
        "Litigation costs associated with challenges of high-type patents"
        field,phiA,phiP,e = x
        k, Beta, t = self.k, self.Beta, self.t
        cs = 0
        int = lambda s: (field.G(self.PiH(field,phiA,phiP,s),s,k)-field.G(field.pi(s,t),s,k))*field.x(s,t)*2*field.LC(s,t)*field.f(s,Beta)
        integral, error = quad(int, self.scc(field,phiA,phiP,e), 1, points=field.int_pts(0,Beta,t))
        return integral

    def W_alt(self,x):
        "Alternative welfare calculation to check that decomposition works"
        field,phiA,phiP,e = x
        return self.I_Lnc(x) + self.I_Lcc(x) + self.I_Hnc(x) + self.I_Hcc(x) - (self.DWL_Lnc(x) + self.DWL_Lcc(x) + self.DWL_Hnc(x) + self.DWL_Hcc(x)) + self.DA_L(x) + self.DA_H(x) - (self.LC_L(x) + self.LC_H(x))


In [ ]:
# set economy-wide parameters

T = 20 # patent length
l0 = 590000 # minimum litigation cost
l1 = 0.15 # rate at which litigation cost increases with value at stake
qH = 1 # high type's win probability in court
qL = 0 # low type's win probability in court
phiA = 16318 # pre-grant fee + attorney costs
phiP = 5985 # post-grant fee
gr0 = 0.675
gr1 = 0.778 # grant rate before and after examiner time is doubled (from Frakes & Wasserman)
bdgt = 4130 # patent office examination budget per application
neg_fee = 1 # equals 1 if negative fixed fees are allowed
dist = 1 # distribution of cost reduction s. possible values: 0 = frechet or 1 = loglogistic
negk0 = 0 # if equal to 1, mean of R&D cost is adjusted such that it converges to k0 + k1*s as s grows, where k0 can be either positive or negative
num = 0 # 0 = count only inventions on which patents are applied for in TFP and R&D, 1 = count all inventions
denom = 0 # 0 = count only inventions on which patents are applied for in denominator of TFPI and RPA, 1 = count all inventions


### Aggregate level

para = np.array([550928.677, 84460.4408629642, 10.55, 1.8, 0.15, 0.712, 0.0238, 0.576, 721817.6309392679, 0.000024828402544995])
x0 = 0.4, 1e5, 3e10, 0.85, 5e-07

field = Field(*para)

# 5-eq model with e, k0, k1, alpha and beta as free parameters, and GR, LR, WR, RPA, TFPI as targets

# run least-squares minimization to generate starting values for exact solver
weights = [1,1,1,1,1]
solution = field.estimation_lq(x0,weights)
x0 = solution.x

# run exact solver
sol_flat = field.estimation(x0)
sol = sol_flat[0], [sol_flat[1], sol_flat[2], 0], [sol_flat[3], sol_flat[4]], 0
e, k, Beta, t = sol
sol

In [ ]:
ql_p, qh_p = field.q(sol, prepon=1)
ql_c, qh_c = field.q(sol, prepon=0)
#print(ql_p, qh_p, ql_c, qh_c)


# estimate examination cost function
def eqs(x):
    gamma0 = x[0]
    gamma1 = x[1]
    Lambda = field.SHA(sol)
    e0 = (1-gr0)/(1-Lambda)
    e1 = (1-gr1)/(1-Lambda)
    return np.array([field.gamma(gamma0,gamma1,e0)-2*field.gamma(gamma0,gamma1,e1), field.gamma(gamma0,gamma1,e)-bdgt]) # (phiA-15000+field.gr*phiP)

gam0 = 2, 10
gam = optimize.root(eqs, gam0, method='hybr')
gamma0, gamma1 = gam.x

#gamma1 = eta*((1-e)*field.AL(e,k,Beta,t)+field.AH(e,k,Beta,t))/((1-e)*field.AL(e,k,Beta,t)*np.log(1-e))
#omega = (phiA-15000 + phiP * field.gr)/(-np.log(1-e))**(1/gamma1)

# Parameterize welfare analysis

welf = Welfare(*[k,Beta,t,gam.x])

# counterfactuals

# fee revenue and number of applications in baseline

FR = field.AL(e,k,Beta,t)*(phiA-15000 + (1-e)*phiP) + field.AH(e,k,Beta,t)*(phiA-15000+phiP) # fee revenue before frontloading
A0 = field.AL(e,k,Beta,t)+field.AH(e,k,Beta,t)


# frontload fees and raise e (revenue neutral)

phiA = phiA+phiP
phiP = 0

def rc(z):
    "Set old budget plus difference in fee revenue equal to examination cost"
    return A0*bdgt - FR + (field.AL(z,k,Beta,t)+field.AH(z,k,Beta,t))*(phiA-15000-field.gamma(gamma0,gamma1,z))
root = optimize.root(rc,x0=e)
e_fl = root.x[0]

# optimal combination of phiA and e

neg_fee = 0

def objective(x):
    phiA, e = x
    return -(field.W(phiA,0,e,k,Beta,t)-field.Gamma(phiA,0,e,k,Beta,t,gamma0,gamma1))

def constraint(x):
    "phiA, e such that low types randomize over license fee"
    phiA, e = x
    phiP = 0
    def lambdalow(phiA,phiP,s,e,k,t):
        return (field.G(field.pi(s,t)+(1-max(0,field.x(s,t))*(1-qH))*field.DeltaH(s,t)-max(0,field.x(s,t))*field.LC(s,t)-phiA-phiP,s,k)-field.G(field.pi(s,t),s,k))/(field.G(field.pi(s,t)+(1-max(0,field.x(s,t))*(1-qH))*field.DeltaH(s,t)-max(0,field.x(s,t))*field.LC(s,t)-phiA-phiP,s,k)-e*field.G(field.pi(s,t),s,k))
    def scc(phiA,phiP,e,k,t):
        def cc(s):
            return (1-qL-lambdalow(phiA,phiP,s,e,k,t)*(qH-qL))*(4/5)*field.DeltaH(s,t)-field.LC(s,t)
        root = optimize.root(cc,x0=1)
        return root.x[0] if root.success == True else 1 #'Error: root finding algorithm did not succeed'
    return (1-e)*(5/4)*field.LC(scc(phiA,phiP,e,k,t),t)-phiA

policy0 = [1e5,0.7]
cons = ({'type': 'ineq', 'fun': constraint})
bnds = ((0,None),(0,1))

optimum = optimize.minimize(objective,policy0,method='Nelder-Mead')  #method='SLSQP',bounds=bnds,constraints=cons)
phi_o, e_o = optimum.x

# reset fees and neg_fee to baseline level

phiA = 16318
phiP = 5985
neg_fee = 1

counterfac = []

# parameters for counterfactuals to run

counterfactuals = [['Baseline', e, phiA, phiP, l0, l1, qH, qL, neg_fee], ['Registration system', 0, phiA, phiP, l0, l1, qH, qL, neg_fee], ['Frontload fees', e, phiA+phiP, 0, l0, l1, qH, qL, neg_fee], ['Frontload fees and raise e', e_fl, phiA+phiP, 0, l0, l1, qH, qL, neg_fee], ['Perfect PTAB', e, phiA, phiP, 350000, 0, qH, qL, neg_fee], ['Imperfect PTAB (Preponderance of Evidence)', e, phiA, phiP, 350000, 0, qh_p, ql_p, neg_fee], ['Imperfect PTAB (Clear and Convincing Evidence)', e, phiA, phiP, 350000, 0, qh_c, ql_c, neg_fee], ['No negative fixed fees', e, phiA, phiP, l0, l1, qH, qL, 0], ['Optimal policy', e_o, phi_o, 0, l0, l1, qH, qL, 0]]

for i in range(len(counterfactuals)):
    counterfactual, e_cf, phiA_cf, phiP_cf, l0_cf, l1_cf, qH_cf, qL_cf, neg_fee_cf = counterfactuals[i]
    sol = e_cf, k, Beta, t
    e = e_cf
    phiA = phiA_cf
    phiP = phiP_cf
    l0 = l0_cf
    l1 = l1_cf
    qH = qH_cf
    qL = qL_cf
    neg_fee = neg_fee_cf
    output = [[T,l0,l1,qH,qL,phiA,phiP,gr0,gr1,bdgt,neg_fee,dist,num,denom],para,[e,k[0],k[1],Beta[0],Beta[1]],[field.s_hat(0,t),field.s_hat(e,t),field.s_cc(e,k,t),field.PiH(field.s_cc(e,k,t),t)-field.pi(field.s_cc(e,k,t),t)],field.totals(sol),field.targets(sol),[field.SHA(sol),field.SHA_nc(sol),field.SHA_cc(sol),field.SHP(sol),field.SHP_nc(sol),field.SHP_cc(sol),field.SP_cc(e,k,Beta,t),field.LR_cc(e,k,Beta,t),field.xbar(sol),field.ybar(sol),field.DeltaH(field.s_cc(e,k,t),t),field.LC(field.s_cc(e,k,t),t),field.FR(e,k,Beta,t),field.FR(e,k,Beta,t)/(field.AL(e,k,Beta,t)+field.AH(e,k,Beta,t)),gamma0,gamma1,field.gamma(gamma0,gamma1,0),field.gamma(gamma0,gamma1,e),field.Gamma(phiA,phiP,e,k,Beta,t,gamma0,gamma1)],[field.W(phiA,phiP,e,k,Beta,t),field.W(phiA,phiP,e,k,Beta,t)-field.Gamma(phiA,phiP,e,k,Beta,t,gamma0,gamma1),field.CS(phiA,phiP,e,k,Beta,t),field.CS(phiA,phiP,e,k,Beta,t)-field.Gamma(phiA,phiP,e,k,Beta,t,gamma0,gamma1)],[welf.I_Lnc([field,phiA,phiP,e]), welf.I_Lcc([field,phiA,phiP,e]), welf.I_Hnc([field,phiA,phiP,e]), welf.I_Hcc([field,phiA,phiP,e]), welf.DWL_Lnc([field,phiA,phiP,e]), welf.DWL_Lcc([field,phiA,phiP,e]), welf.DWL_Hnc([field,phiA,phiP,e]), welf.DWL_Hcc([field,phiA,phiP,e]), welf.DA_L([field,phiA,phiP,e]), welf.DA_H([field,phiA,phiP,e]), welf.LC_L([field,phiA,phiP,e]), welf.LC_H([field,phiA,phiP,e]), welf.W([field,phiA,phiP,e])-welf.Gamma([field,phiA,phiP,e])-welf.I_Lnc([field,phiA,phiP,e])-welf.I_Lcc([field,phiA,phiP,e])]]
    output_flat = []
    output_flat.append(counterfactual)
    for sublist in output:
        for item in sublist:
            output_flat.append(item)
    counterfac.append(output_flat)


counterfac = np.array(counterfac)
counterfac.shape = (len(counterfactuals),len(output_flat))
counterfac = counterfac.tolist()
counterfac